# HANA Cloud Vector Store

>HANA Cloud Vector Store is a vector store fully integrated into the HANA Cloud database.

It supports:
- TODO1
- TODO2

This notebook shows how to use the HANA Cloud Vector Store.

Installation of the HANA database driver.

In [1]:
# Pip install necessary package
%pip install --upgrade --quiet  hdbcli

Note: you may need to restart the kernel to use updated packages.


We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "Your OpenAI API key"

In [4]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.hanavector import HanaDB
from langchain_community.embeddings import OpenAIEmbeddings

In [5]:
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

/home/martin/dev/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
from hdbcli import dbapi

connection = dbapi.connect(
    address=os.environ.get("DB_ADDRESS"),
    port=os.environ.get("DB_PORT"),
    user=os.environ.get("DB_USER"),
    password=os.environ.get("DB_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=False,
)

## Similarity Search with Cosine Simliarity (Default)

In [9]:
db = HanaDB.from_documents(
    embedding=embeddings,
    documents=docs,
    connection=connection,
    table_name = "LANGCHAIN_DEMO"
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [10]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query)

NameError: name 'db' is not defined

In [9]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.18456886638850434
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
----------------------

## Maximal Marginal Relevance Search (MMR)
Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents.

In [10]:
docs_with_score = db.max_marginal_relevance_search_with_score(query)

In [11]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.18453882564037527
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
----------------------

## Working with the vectorstore

In [8]:
store = HanaDB(
    connection=connection,
    embedding_function=embeddings,
)

### Add documents
We can add documents to the existing vectorstore.

In [19]:
store.add_documents([Document(page_content="foo")])

['048c2e14-1cf3-11ee-8777-e65801318980']

In [20]:
docs_with_score = db.similarity_search_with_score("foo")

In [21]:
docs_with_score[0]

(Document(page_content='foo', metadata={}), 3.3203430005457335e-09)

In [22]:
docs_with_score[1]

(Document(page_content='A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  \n\nWe’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. \n\nWe’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.', metadata={'source': '../../../state_of_the_union

### Using a VectorStore as a Retriever (e.g. in chains using RAG)

In [26]:
retriever = store.as_retriever()

In [27]:
print(retriever)

tags=None metadata=None vectorstore=<langchain_community.vectorstores.pgvector.PGVector object at 0x29f94f880> search_type='similarity' search_kwargs={}
